# Parametric Waveform Analyser

This instrument consists of two extended multiplexers; one for readout and one for drive. They share a parametric sequencer and the parameters from each are used to build a context dictionary to set what is uploaded to the aw. The drive multiplexer sidebands one microwave source and the readout multiplexer sidebands a heterodyne microwave source. The settings of an alazar card are updated based on what is running on the sequencer and the preferences of the readout multiplexer. Template elements are loaded from yaml files.

In [17]:
import numpy as np
import os
import yaml
from pprint import pprint
from lomentum.loader import read_element
from lomentum.plotting import plotter
from lomentum.loader import register_user_module
from qdev_wrappers.pulse_building import atoms_ext 
from qcodes.instrument.base import InstrumentBase
from qdev_wrappers.station_configurator import StationConfigurator
from qdev_wrappers.alazar_controllers.ATSChannelController import ATSChannelController
from qdev_wrappers.customised_instruments.interfaces.microwave_source_interface import SimulatedMicrowaveSourceInterface, SGS100AMicrowaveSourceInterface, SGS100AExternalMicrowaveSourceInterface
from qdev_wrappers.customised_instruments.interfaces.AWG_interface import SimulatedAWGInterface, AWG5014Interface
from qdev_wrappers.customised_instruments.composite_instruments.parametric_sequencer.parametric_sequencer import ParametricSequencer
from qdev_wrappers.customised_instruments.composite_instruments.parametric_waveform_analyser.pwa import ParametricWaveformAnalyser
from qdev_wrappers.customised_instruments.composite_instruments.heterodyne_source.heterodyne_source import OneSourceHeterodyneSource, SimulatedHeterodyneSource
register_user_module(atoms_ext)

In [15]:
def print_params(*instruments):
    for instr in instruments:
        print('\n')
        print(instr.name.upper())
        for k, v in instr.parameters.items():
            if k != 'IDN':
                print(k, ': ', v())
        for k, v in instr.submodules.items():
            if isinstance(v, InstrumentBase):
                print_params(v)

In [19]:
real_instruments_present = True

In [20]:
try:
    Instrument.remove_instance(ps)
    Instrument.remove_instance(awg_if)
    Instrument.remove_instance(carrier_if)
    Instrument.remove_instance(awg_instrument)
    Instrument.remove_instance(microwave_source_instrument)
except NameError:
    pass

if real_instruments_present:
    sc = StationConfigurator()
    awg_instrument = sc.load_instrument('awg')
    drive_microwave_source_instrument = sc.load_instrument('microwave_source')
    readout_microwave_source_instrument = sc.load_instrument('microwave_source_dual')
    alazar_instrument = sc.load_instrument('alazar')
    awg_if = AWG5014Interface('awg_if', awg_instrument)
    drive_microwave_source_if = SGS100AExternalMicrowaveSourceInterface('drive_microwave_source_if',
                                                                        drive_microwave_source_instrument)
    readout_microwave_source_if = SGS100AMicrowaveSourceInterface('readout_microwave_source_if',
                                                                  readout_microwave_source_instrument)
    heterodyne_source = OneSourceHeterodyneSource('heterodyne_source', readout_microwave_source_if)
else:
    awg_if = SimulatedAWGInterface('awg_if')
    drive_microwave_source_if = SimulatedMicrowaveSourceInterface('drive_microwave_source_if')
    heterodyne_source = SimulatedHeterodyneSource('heterodyne_source')
    raise RuntimeError('Simulte alazar before you go further!')

ps = ParametricSequencer('ps', awg_if)
alazar_controller = ATSChannelController(name='alazar_controller', alazar_name='alazar')
ps.sequence_mode('element')

# create multiplexer with two sidebanders.
pwa = ParametricWaveformAnalyser('pwa', ps, alazar, alazar_controller, heterodyne_source, drive_source)

FileNotFoundError: [Errno 2] No such file or directory: 'instrument_config.yml'

### What is missing:

- Ability to swap out alazar + heterodyne source for MIDAS
- Ability to swap out tektronix AWG5014C for 4DSP system
- Ability to simulate alazar